In [1]:
import math
import time
import math
import osmnx as ox
import numpy as np
import networkx as nx
import geopandas as gpd
from shapely.geometry import Point
from shapely.geometry import LineString
# from distance import nearest_edges as getNE

In [2]:
# test
# G1 = ox.graph_from_place('Chicago, Cook County, Illinois, United States', network_type='drive') # full

In [3]:
# fig, ax = ox.plot_graph(G1)
# G1.nodes[39076461]

In [4]:
# Gp1 = ox.project_graph(G1) # need to do projection before consolidation, do projection also for query points!
# Gc1 = ox.consolidate_intersections(Gp1, tolerance=20, rebuild_graph=True)

In [5]:
# Gc1.nodes[0]

In [2]:
def PrepareNodeidDict(G):
    
    '''
    This function is used to handle the 'id-0, id-1, id-2..' scenario appeared in nodeid after applying consolidate_intersection
    Parameter:
    G: after applying consolidate_intersection
    '''
    
    nodeid_dict = {}
    
    new_id = 0
    for origin_id in G:
        if type(origin_id) == int:
            nodeid_dict.update({origin_id:origin_id})
            new_id = origin_id
        else:
            new_id += 1
            nodeid_dict.update({origin_id:new_id})
            
    return nodeid_dict

In [3]:
# Translate the edges in the graph into format:
#   - edge_id, node1_id, node2_id, length
#
# Parameters:
# - G: the OSMnx road graph, projected, consolidated
# - path_to: save the edge format to
#
# Return:
# - edge dictionary?
#

def ProcessEdges(G, path_to, nodeid_dict):
    
    edge_dict = {}
    edge_list = []
    edge_np = np.array(G.edges)
    
    for i, edge in enumerate(G.edges):
        
        if i % 10000 == 0:
            print("current edge: ", i)
            
        # using the origin node format for calculation
        origin_node1_id = edge[0]
        origin_node2_id = edge[1]
        #length_meters = nx.shortest_path_length(G, source=origin_node1_id, target=origin_node2_id, weight='length')
        length_meters = G[origin_node1_id][origin_node2_id][0]['length']
        
        # used the save node format for recording
        new_node1_id = nodeid_dict[origin_node1_id]
        new_node2_id = nodeid_dict[origin_node2_id]
        
        # the projected location is newly added, # edge_id, node1_id, node2_id, length in meters
        edge_list.append([i, new_node1_id, new_node2_id, length_meters, G.nodes[origin_node1_id]['x'], G.nodes[origin_node1_id]['y'], G.nodes[origin_node2_id]['x'], G.nodes[origin_node2_id]['y']]) 
        
        edge_dict.update({(new_node1_id, new_node2_id):i}) # using 2 node_id as edge's key, using edge id as value
        edge_dict.update({(new_node2_id, new_node1_id):-i}) # indicate the inverse
        if i == 0:
            edge_dict.update({(new_node2_id, new_node1_id):-0.1}) # special case for zero
        
    edge_list_np = np.array(edge_list)
    np.savetxt(path_to, edge_list_np, delimiter=',')
    
    edge_list_np = edge_list_np[:,0:4] # we don't need the remaining info
    
    return edge_dict

In [4]:
def EuclideanDist(p1x, p1y, p2x, p2y):
    return math.sqrt((p1x - p2x)**2 + (p1y - p2y)**2)

In [5]:
def GetDistancesOptimized(G, path_from, path_to, nodeid_dict):
    
    '''
    Parameters:
    @path_from: the query file
    @G: projected, consolidated road network
    '''
    # get the nodes stored in csv 
    nodes = np.genfromtxt(path_from, delimiter=' ')

    
    longitudes = nodes[:,0]
    latitudes = nodes[:,1]

    
    points_list = [Point((lng, lat)) for lat, lng in zip(latitudes, longitudes)] # turn into shapely geometry
    points = gpd.GeoSeries(points_list, crs='epsg:4326') # turn into geoseries, with default crs, i.e., ox.settings.default_crs
    points_proj = points.to_crs(G.graph['crs'])
    
    xs = [pp.x for pp in points_proj]
    ys = [pp.y for pp in points_proj]

   
    
    # find the nearest edges for all query points
    nearest_edges = ox.distance.get_nearest_edges(G, xs, ys, method='kdtree', dist=1) # dist = 1 for meters
    # print("xs: ", xs)
    # print("ys: ", ys)
    # nearest_edges = getNE(G, xs, ys, interpolate=1000.0)
    print("finished finding all the nearest edges")
    
    distances = []
    
    # find distances
    for i in range(len(nodes)):
        if i % 10000 == 0:
            print("current point: ", i)
            
        # edge's 2 ends's node id
        point1_id = nearest_edges[i][0] # the nearest edge'S first end's node id
        point2_id = nearest_edges[i][1] # the nearest edge'S second end's node id

        # generate projection on nearest edge
        #ref_point = Point(nodes[i,1], nodes[i,0]) # Point(lon, lat)
        ref_point = Point(xs[i], ys[i]) # the projected point
        
        point_1 = Point(G.nodes[point1_id]['x'], G.nodes[point1_id]['y']) # x is longitude, y is latitude
        point_2 = Point(G.nodes[point2_id]['x'], G.nodes[point2_id]['y'])

        # this is the distance from one end to the projected position (but we don't know the other end)
        projected_dist = LineString([point_1, point_2]).project(ref_point)
        # print("projected_dist: ", projected_dist)


        # get the projected point
        projected_point = list(LineString([point_1, point_2]).interpolate(projected_dist).coords)
        # print("projected_point: ", projected_point)
    
            
        
        # using Haversine(OSMnx's)
        #distance_1 = ox.distance.great_circle_vec(projected_point[0][1], projected_point[0][0], G.nodes[point1_id]['y'], G.nodes[point1_id]['x'])
        #distance_2 = ox.distance.great_circle_vec(projected_point[0][1], projected_point[0][0], G.nodes[point2_id]['y'], G.nodes[point2_id]['x'])
        distance_1 = EuclideanDist(projected_point[0][0], projected_point[0][1], G.nodes[point1_id]['x'], G.nodes[point1_id]['y'])
        distance_2 = EuclideanDist(projected_point[0][0], projected_point[0][1], G.nodes[point2_id]['x'], G.nodes[point2_id]['y'])
        
        # used the save node format for recording
        new_node1_id = nodeid_dict[point1_id]
        new_node2_id = nodeid_dict[point2_id]
        
        # the query node's closest edge's two nodes and its distance
        distances.append([new_node1_id, new_node2_id, distance_1, distance_2])
    
    distances_np = np.array(distances)
    np.savetxt(path_to, distances_np, delimiter=',')
    
    return distances_np

In [6]:
# convert the distance into edison's point format:
#  - edge_id, distance_1, distance_2
#
# Parameters:
#  - distances: the ourput results from GetDistances()
#  - edge_dict: the dictionary from ProcessEdges()
#  - path_to: save path
#
# Return:
#  - the processed PointFormat
#
def ConvertFormat(distances, edge_dict, path_to):
    
    processed_distances = []
    
    for i in range(len(distances)):
        edge_node_pair = (distances[i][0], distances[i][1])
        edge_key = edge_dict[edge_node_pair]
        
        if edge_key >= 0:
            processed_distances.append([edge_key, distances[i][2], distances[i][3]])
        else:
            # change the order
            edge_key = int(abs(edge_key)) # process the edge_key
            processed_distances.append([edge_key, distances[i][3], distances[i][2]])
                
    processed_distances_np = np.array(processed_distances)
    np.savetxt(path_to, processed_distances_np, delimiter=',')
    
    return processed_distances_np

In [7]:
# === execution (New York) ===

# G1 = ox.graph_from_place('London, Greater London, England, United Kingdom', network_type='drive') # full

# G1 = ox.graph_from_place('Chicago, Cook County, Illinois, USA', network_type='drive') # full

# G1 = ox.graph_from_place('Detroit, Wayne County, Michigan, USA', network_type='drive') # full

G1 = ox.graph_from_place('San Francisco, California, USA', network_type='drive') # full

Gp1 = ox.project_graph(G1) # need to do projection before consolidation, do projection also for query points!

In [8]:
# tolerance of London=46, and 20 for other datasets!!!!
Gc1 = ox.consolidate_intersections(Gp1, tolerance=20, rebuild_graph=True) 
nodeid_dict1 = PrepareNodeidDict(Gc1)

In [9]:
print("node num: ", Gc1.number_of_nodes())
print("edge num: ", Gc1.number_of_edges())

node num:  16288
edge num:  51845


In [10]:
city_name = "Detroit" # London_cut, Chicago, Detroit, San_Francisco

In [11]:
print("node num: ", Gc1.number_of_nodes())
print("edge num: ", Gc1.number_of_edges())
edge_dict1 = ProcessEdges(Gc1, "results/"+city_name+"_clean.edge", nodeid_dict1)

node num:  16288
edge num:  51845
current edge:  0
current edge:  10000
current edge:  20000
current edge:  30000
current edge:  40000
current edge:  50000


In [12]:
for sample_ratio in [1.0]: #[0.25, 0.5, 0.75, 1.0]:
    start_time = time.time()

    # query_set1 = "Datasets_Yun/"+city_name+"_clean_s"+str(sample_ratio)+".csv"
    # distances1 = GetDistancesOptimized(Gc1, query_set1, "results/"+city_name+"_clean_s"+str(sample_ratio)+".distances", nodeid_dict1)

    query_set1 = "Datasets_Yun/"+city_name+"_clean.csv"
    distances1 = GetDistancesOptimized(Gc1, query_set1, "results/"+city_name+"_clean.distances", nodeid_dict1)
    
    # processed_distance1 = ConvertFormat(distances1, edge_dict1, "results/"+city_name+"_clean_s"+str(sample_ratio)+".processed.distances")
    print("--- %s seconds ---" % (time.time() - start_time))

pos [ 134515 3968722 6459035 ... 2178924 1196574 4765979]
finished finding all the nearest edges
current point:  0
current point:  10000
current point:  20000
current point:  30000
current point:  40000
current point:  50000
current point:  60000
current point:  70000
current point:  80000
current point:  90000
current point:  100000
current point:  110000
current point:  120000
current point:  130000
current point:  140000
current point:  150000
current point:  160000
current point:  170000
current point:  180000
current point:  190000
current point:  200000
current point:  210000
current point:  220000
current point:  230000
current point:  240000
current point:  250000
current point:  260000
current point:  270000
current point:  280000
current point:  290000
current point:  300000
current point:  310000
current point:  320000
current point:  330000
current point:  340000
current point:  350000
current point:  360000
current point:  370000
current point:  380000
current point:  3900

In [14]:
# # === execution (Atlanta) ===
# start_time = time.time()
# # place2_road_full = ox.graph_from_place('Atlanta, Georgia', network_type='drive') # full, this do not contains the query points

# # bounding box for City of Johns Creek, 
# bbox = (34.089791, 33.990863, -84.097653, -84.281025) # latitude high, latitude low, longitude high, longitude low
# G2 = ox.graph_from_bbox(*bbox, network_type='drive')
# Gp2 = ox.project_graph(G2) # need to do projection before consolidation, do projection also for query points!
# Gc2 = ox.consolidate_intersections(Gp2, tolerance=20, rebuild_graph=True)

# nodeid_dict2 = PrepareNodeidDict(Gc2)
# edge_dict2 = ProcessEdges(Gc2, "results/Atlanta_edges", nodeid_dict2)
# query_set2 = "Datasets_Zhe/Atlanta_police_calls_ours"
# distances2 = GetDistancesOptimized(Gc2, query_set2, "results/Atlanta_distances", nodeid_dict2)
# processed_distance2 = ConvertFormat(distances2, edge_dict2, "results/Atlanta_processed_distances")
# print("--- %s seconds ---" % (time.time() - start_time))

In [15]:
# # === execution (Los Angeles) ===
# start_time = time.time()
# G3 = ox.graph_from_place('Los Angeles, California', network_type='drive') # full
# Gp3 = ox.project_graph(G3) # need to do projection before consolidation, do projection also for query points!
# Gc3 = ox.consolidate_intersections(Gp3, tolerance=20, rebuild_graph=True)
# del G3
# del Gp3
# nodeid_dict3 = PrepareNodeidDict(Gc3)
# edge_dict3 = ProcessEdges(Gc3, "results/Los_Angeles_edges", nodeid_dict3)
# query_set3 = "Datasets_Zhe/Los_Angles_crime_ours"
# distances3 = GetDistancesOptimized(Gc3, query_set3, "results/Los_Angeles_distances", nodeid_dict3)
# processed_distance3 = ConvertFormat(distances3, edge_dict3, "results/Los_Angeles_processed_distances")
# print("--- %s seconds ---" % (time.time() - start_time))

In [16]:
# # === execution (Seattle) ===
# start_time = time.time()
# G4 = ox.graph_from_place('Seattle, Washington', network_type='drive')  # full
# Gp4 = ox.project_graph(G4) # need to do projection before consolidation, do projection also for query points!
# Gc4 = ox.consolidate_intersections(Gp4, tolerance=20, rebuild_graph=True)
# nodeid_dict4 = PrepareNodeidDict(Gc4)
# edge_dict4 = ProcessEdges(Gc4, "results/Seattle_edges", nodeid_dict4)
# query_set4 = "Datasets_Zhe/Seattle_crime_ours"
# distances4 = GetDistancesOptimized(Gc4, query_set4, "results/Seattle_distances", nodeid_dict4)
# processed_distance4 = ConvertFormat(distances4, edge_dict4, "results/Seattle_processed_distances")
# print("--- %s seconds ---" % (time.time() - start_time))